## Hitting vs Defense Plot


In [2]:
import pybaseball as pb
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
import plotly.graph_objects as go
import os
from PIL import Image

In [3]:
''' Going to use 2021 batting stats with a minimum of 200 PA'''
df = pb.batting_stats(2021, qual=200)
df

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
17,16252,2021,Trea Turner,- - -,28,148,595,646,195,130,...,0.074,112.2,225,0.460,489,0.160,0.262,0.303,0.484,0.362
2,19611,2021,Vladimir Guerrero Jr.,TOR,22,161,604,698,188,110,...,0.151,117.4,274,0.552,496,0.127,0.250,0.306,0.591,0.416
0,11579,2021,Bryce Harper,PHI,28,141,488,599,151,73,...,0.181,116.3,177,0.492,360,0.119,0.263,0.301,0.610,0.430
1,20123,2021,Juan Soto,WSN,22,151,502,654,157,106,...,0.133,116.6,217,0.524,414,0.200,0.263,0.304,0.544,0.430
42,12533,2021,Marcus Semien,TOR,30,162,652,724,173,87,...,0.098,109.5,210,0.413,509,0.158,0.245,0.244,0.453,0.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,12977,2021,Michael Perez,PIT,28,70,210,231,30,14,...,0.063,106.2,46,0.324,142,0.166,0.276,0.171,0.302,0.244
356,14885,2021,Pat Valaika,BAL,28,91,259,281,52,39,...,0.048,109.7,61,0.326,187,0.150,0.276,0.213,0.337,0.266
354,24655,2021,Owen Miller,CLE,24,60,191,202,39,27,...,0.058,108.2,45,0.326,138,0.180,0.296,0.222,0.342,0.264
314,12907,2021,Gregory Polanco,PIT,29,107,336,382,70,45,...,0.096,116.2,115,0.479,240,0.153,0.284,0.231,0.409,0.315


### High Level EDA

In [4]:
df.dtypes

IDfg        int64
Season      int64
Name       object
Team       object
Age         int64
           ...   
CStr%     float64
CSW%      float64
xBA       float64
xSLG      float64
xwOBA     float64
Length: 319, dtype: object

In [5]:
df.describe()
# Looks like the mean values would be good to have as a comparison against

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
count,362.000000,362.0,362.000000,362.000000,362.000000,362.000000,362.000000,362.000000,362.000000,362.000000,...,362.000000,362.000000,362.000000,362.000000,362.000000,362.000000,362.000000,362.000000,362.000000,362.000000
mean,13981.218232,2021.0,28.439227,114.723757,380.127072,426.872928,96.251381,60.494475,19.196133,1.627072,...,0.083362,111.119061,115.198895,0.394254,290.212707,0.160948,0.272384,0.246638,0.415790,0.321605
std,4896.057822,0.0,3.469361,29.897534,132.132815,149.539183,40.187584,26.795134,8.950321,1.644814,...,0.043356,3.120382,51.316024,0.076797,110.941733,0.029687,0.027949,0.026922,0.071599,0.036815
min,1177.000000,2021.0,20.000000,48.000000,174.000000,202.000000,30.000000,14.000000,3.000000,0.000000,...,0.000000,100.600000,30.000000,0.138000,101.000000,0.080000,0.140000,0.170000,0.258000,0.216000
25%,11348.500000,2021.0,26.000000,89.000000,260.500000,291.250000,60.000000,38.000000,12.000000,0.000000,...,0.051000,109.200000,72.000000,0.340000,190.750000,0.142000,0.255000,0.230000,0.367250,0.296250
50%,14558.500000,2021.0,28.000000,116.500000,367.500000,408.500000,91.500000,58.500000,18.000000,1.000000,...,0.079000,110.800000,106.000000,0.400000,280.500000,0.162500,0.270000,0.248000,0.409500,0.319500
75%,17754.250000,2021.0,31.000000,142.000000,497.000000,554.750000,128.750000,79.000000,26.000000,2.000000,...,0.108000,113.200000,152.000000,0.445500,381.000000,0.181000,0.293000,0.264750,0.456750,0.344000
max,27506.000000,2021.0,41.000000,162.000000,664.000000,724.000000,195.000000,136.000000,42.000000,8.000000,...,0.243000,122.200000,274.000000,0.579000,573.000000,0.250000,0.345000,0.319000,0.618000,0.430000


## PLotting wRC+ against def to see isolate the well rounded players
#### Can't find OAA so going with Def

In [6]:
batting_df = pb.batting_stats(2021, qual=200)

In [7]:
plot_df = (
    batting_df
     .pipe(lambda df: df[["IDfg", "Name", "Team", 'G', 'PA', 'OPS', 'Dol', 'Spd','Off', 'WAR', 'wRC+', 'Def']]) # just getting the columns we need
)

In [8]:
plot_df["wRC+"].describe()

count    362.000000
mean     100.701657
std       25.289557
min       35.000000
25%       85.000000
50%      100.000000
75%      115.000000
max      170.000000
Name: wRC+, dtype: float64

In [9]:
plot_df["Def"].describe()

count    362.000000
mean      -0.629834
std        6.839704
min      -18.100000
25%       -5.100000
50%       -0.900000
75%        3.975000
max       19.100000
Name: Def, dtype: float64

In [10]:
# Find top 25th percentile in both hitting and defense
good_players = plot_df.loc[(plot_df['wRC+'] >= 115) & (plot_df['Def'] >= 4)]

In [11]:
# Find the bottom 25th percentile players in wRC+ and Def
bad_players = plot_df.loc[(plot_df['wRC+'] <= 85) & (plot_df['Def'] <= -5)]

In [12]:
fig1 = px.scatter(
    plot_df, x="Def", y="wRC+",
    color="WAR",
    hover_name="Name",
    hover_data=["PA"],
    color_continuous_scale=px.colors.diverging.Tropic,
    width=1800, height=800,

)

fig1.update_traces(marker_line_width=2, marker_size=10)
fig1.update_yaxes(nticks=20)
fig1.update_xaxes(nticks=40)

# Good hitting and defense
fig1.add_shape(
    x0=plot_df["Def"].quantile(q=0.8), 
    y0=plot_df["wRC+"].quantile(q=0.8),
    x1=plot_df["Def"].quantile(q=1),
    y1=plot_df["wRC+"].quantile(q=1),
    line=dict(color="LightSeaGreen", width=2, dash="dot"),
    layer='below'
)
fig1.add_annotation(
    x=plot_df["Def"].quantile(q=0.99),
    y=plot_df["wRC+"].quantile(q=.99),
    text="Great Hitting and Defense",
    showarrow=False,
    font=dict(
        family="arial",
        size=18,
        color="black"
    )
    )

# Bad hitting and defense
fig1.add_shape(
    x0=plot_df["Def"].quantile(q=0), 
    y0=plot_df["wRC+"].quantile(q=0),
    x1=plot_df["Def"].quantile(q=0.2),
    y1=plot_df["wRC+"].quantile(q=0.2),
    line=dict(color="#EF553B", width=2, dash="dot")
)
fig1.add_annotation(
    x=plot_df["Def"].quantile(q=0.01), 
    y=plot_df["wRC+"].quantile(q=0.01),
    text="Terrible Hitting and Defense",
    showarrow=False,
    font=dict(
        family="arial",
        size=18,
        color="black"
    )
    )


# Medians
fig1.add_hline(y=np.median(df["wRC+"]), line_dash="dot", row="all", col="all",
              annotation_text="Median wRC+", 
              annotation_position="bottom right")

fig1.add_vline(x=np.median(df["Def"]), line_dash="dot", row="all", col="all",
              annotation_text="Median Def", 
              annotation_position="bottom right")


def add_player_image(player_name:str, x_adjustment:int, y_adjustment:int):
    image = Image.open(f'notable_faces/{player_name}.jpg')
    fig1.add_layout_image(dict(
        source=image, 
        xref='x',
        yref='y',
        x=int(plot_df.loc[(plot_df['Name'] == f'{player_name}')]["Def"] + x_adjustment), 
        y=int(plot_df.loc[(plot_df['Name'] == f'{player_name}')]["wRC+"] + y_adjustment),
        sizex=15,
        sizey=30, 
        layer='above'
    ))
# Adding Notable PLayer faces
# Buxton
add_player_image('Byron Buxton', 2, 10)
# Polanco
add_player_image('Gregory Polanco', -4, 5)
#Zunino
add_player_image('Mike Zunino', 1, 20)
# Kelenic
add_player_image('Jarred Kelenic', -2, -5)
# Sano
add_player_image('Miguel Sano', -1, 35)
# Murphy
add_player_image('Sean Murphy', -1, -5)
# Harper
add_player_image('Bryce Harper', -3, 10)
# Ohtani
add_player_image('Shohei Ohtani', -3, 20)


fig1
